In [2]:
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import nltk

nltk.download('punkt')

from pathlib import Path
import sys
import itertools

from pylab import savefig

#!pip install -q tensorflow-model-analysis
from gensim.models import KeyedVectors
#import tensorflow as tf
#import tensorflow_model_analysis as tfma

#===============================================================================
#Tensorflow and Keras 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
 
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dropout, Dense, Flatten, LSTM, \
  SpatialDropout1D, Bidirectional, GlobalMaxPooling1D, Conv2D, MaxPool2D,\
  MaxPooling1D, Conv1D, Input, Reshape,Concatenate, Embedding, BatchNormalization,\
  GRU

from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping,\
    ReduceLROnPlateau

from keras.models import Model

from keras.metrics import MeanIoU

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant

from keras import initializers, regularizers, constraints, optimizers, layers, callbacks

#===============================================================================
import sklearn.metrics as skm

from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score,precision_score,recall_score,jaccard_score, hamming_loss, accuracy_score 
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import IterativeStratification
from skmultilearn.problem_transform import LabelPowerset

import os
import datetime


import pandas as pd

import numpy as np
from numpy import array

import re

import matplotlib.pyplot as plt

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
#printmd('**bold**')

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import nltk

print(tf.__version__)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Belldandy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Using TensorFlow backend.
2.0.0


In [3]:
import re
id_stopword_dict = pd.read_csv('stopword.csv', header=None)
id_stopword_dict = id_stopword_dict.rename(columns={0: 'stopword'})

factory = StemmerFactory()
stemmer = factory.create_stemmer()

alay_dict = pd.read_csv('new_kamusalay.csv', encoding='latin-1', header=None)
alay_dict = alay_dict.rename(columns={0: 'original', 
                                      1: 'replacement'})

stopword = 'stopword.txt'
def lowercase(text):
    return text.lower()

def remove_unnecessary_char(text):
    #\xf0\x9f\x98\x84\xf0\x9f\x98\x84\xf0\x9f\x98\x84'
    text = re.sub('\n',' ',text) # Remove every '\n'
    text = re.sub('rt',' ',text) # Remove every retweet symbol
    text = re.sub('xf0',' ',text) # Remove every retweet symbol
    text = re.sub('x9f',' ',text) # Remove every retweet symbol
    text = re.sub('x98',' ',text) # Remove every retweet symbol
    text = re.sub('x82',' ',text) # Remove every retweet symbol
    text = re.sub('x84',' ',text) # Remove every retweet symbol
    text = re.sub('xe2',' ',text) # Remove every retweet symbol
    text = re.sub('x80',' ',text) # Remove every retweet symbol
    text = re.sub('amp',' ',text) # Remove every retweet symbolxa6
    text = re.sub('xa6',' ',text) # Remove every retweet symbol xa4
    text = re.sub('xa4',' ',text) # Remove every retweet symbol

    text = re.sub('user',' ',text) # Remove every username
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) # Remove every URL
    text = re.sub('  +', ' ', text) # Remove extra spaces
    return text
    
def remove_nonaplhanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) 
    return text

alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))
def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

def remove_stopword(text):
    text = ' '.join(['' if word in id_stopword_dict.stopword.values else word for word in text.split(' ')])
    text = re.sub('  +', ' ', text) # Remove extra spaces
    text = text.strip()
    return text
    
    return "".join([""+i if not i.startswith("'") and i not in string.punctuation else i for i in filtered_sentence]).strip()

def stemming(text):
    return stemmer.stem(text)

print("remove_nonaplhanumeric: ", remove_nonaplhanumeric("Halooo,,,,, duniaa!!"))
print("lowercase: ", lowercase("Halooo, duniaa!"))
print("stemming: ", stemming("Perekonomian Indonesia sedang dalam pertumbuhan yang membanggakan"))
print("remove_unnecessary_char: ", remove_unnecessary_char("Hehe\n\n RT USER USER apa kabs www.google.com\n  hehe"))
print("normalize_alay: ", normalize_alay("aamiin adek abis"))
print("remove_stopword: ", remove_stopword("ada hehe adalah huhu yang hehe"))

def preprocess(text):
    text = lowercase(text) # 1
    text = remove_nonaplhanumeric(text) # 2
    text = remove_unnecessary_char(text) # 2
    text = normalize_alay(text) # 3
    text = stemming(text) # 4
    text = remove_stopword(text) # 5
    return text

remove_nonaplhanumeric:  Halooo duniaa 
lowercase:  halooo, duniaa!
stemming:  ekonomi indonesia sedang dalam tumbuh yang bangga
remove_unnecessary_char:  Hehe RT USER USER apa kabs hehe
normalize_alay:  amin adik habis
remove_stopword:  hehe huhu hehe


In [6]:
path_main = "./Model/"


In [7]:
from keras.models import load_model
import pickle
maxlen = 200
tokenizer = Tokenizer(5000)

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

#categories = list(data.columns.values)
#categories = categories[1:]
#print(categories)


main = load_model(path_main+'Main_BiLSTM_Att.h5')
print("Model Main Loaded !")

target = load_model(path_main+'Target_BiLSTM_Att.h5')
print("Model Target Loaded !")

tipe = load_model(path_main+'Type_BiLSTM_Att.h5')
print("Model Tipe Loaded !")

level = load_model(path_main+'Level_BiLSTM_Att.h5')
print("Model Level Loaded !")

#%cd "/content/drive/"

def prediction_to_label_main(prediction):
    categories = ["HS", "Abusive"]
    tag_prob = [(categories[i], prob) for i, prob in enumerate(prediction.tolist())]
    return dict(sorted(tag_prob, key=lambda kv: kv[1], reverse=False))

def prediction_to_label_target(prediction):
    categories = ["HS_Individu", "HS_Group"]
    tag_prob = [(categories[i], prob) for i, prob in enumerate(prediction.tolist())]
    return dict(sorted(tag_prob, key=lambda kv: kv[1], reverse=False))

def prediction_to_label_type(prediction):
    categories = ['HS_Religion', 'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other']
    tag_prob = [(categories[i], prob) for i, prob in enumerate(prediction.tolist())]
    return dict(sorted(tag_prob, key=lambda kv: kv[1], reverse=False))

def prediction_to_label_level(prediction):
    categories = ['HS_Weak', 'HS_Moderate', "HS_Strong"]
    tag_prob = [(categories[i], prob) for i, prob in enumerate(prediction.tolist())]
    return dict(sorted(tag_prob, key=lambda kv: kv[1], reverse=False))

def get_features(text_series):
    """
    transforms text data to feature_vectors that can be used in the ml model.
    tokenizer must be available.
    """
    sequences = tokenizer.texts_to_sequences(text_series)
    return pad_sequences(sequences, maxlen=maxlen, padding='pre')
    
print("======= Notif: Loaded Model Done !========")

Model Main Loaded !
Model Target Loaded !
Model Tipe Loaded !
Model Level Loaded !


In [8]:
def test(f):
  sentiment = []
  f = preprocess(text)
  f =  get_features([f])
  def sa():    
    pred_main = prediction_to_label_main(main.predict(f)[0])
    for label, nilai in pred_main.items():
      if nilai >= 0.5:
        sentiment.append(label)

        pred_target = prediction_to_label_target(target.predict(f)[0])
        for label_target, nilai_target in pred_target.items():
          if nilai_target >= 0.5:
            sentiment.append(label_target)
          else:
            None

        pred_type = prediction_to_label_type(tipe.predict(f)[0])
        for label_tipe, nilai_tipe in pred_type.items():
          if nilai_tipe >= 0.5:
            sentiment.append(label_tipe)
          else:
            None

        pred_level = prediction_to_label_level(level.predict(f)[0])
        for label_level, nilai_level in pred_level.items():
          if nilai_level >= 0.5:
            sentiment.append(label_level)
          else:
            None      

      else:
        sentiment.append("Normal")
    
    return sentiment


  def double_check(temp1):
    for x in temp1:
      temp2 = temp1[:] # create a real copy of temp1
      temp2.remove(x)  # remove x so we won't consider it as dup of itself
      for y in temp2:
          if set(x) == set(y):
              temp1.remove(x)
    
    return hasil

  def triple_check(temp1):
    if len(temp1) <= 1:
      new_list = temp1
    else:
      new_list = list(filter(lambda x : x != "Normal", temp1))
    return new_list

  hasil = sa()
  hasil = double_check(hasil)
  hasil = triple_check(hasil)
  return hasil

In [13]:
text = 'USER USER USER Makanya cebong ber IQ200 sekolam, karena gak bisa bedain fakta dengan provokasi, gak bisa bedain fakta dgn fitnah'

f = preprocess(text)
f =  get_features([f])
hasil = test(text)
print(hasil)

['HS', 'HS_Group', 'HS_Other', 'HS_Strong']


In [12]:

pred_main = prediction_to_label_main(main.predict(f)[0])
print(pred_main)

pred_target = prediction_to_label_target(target.predict(f)[0])
print(pred_target)

pred_type = prediction_to_label_type(tipe.predict(f)[0])
print(pred_type)

pred_level = prediction_to_label_level(level.predict(f)[0])
print(pred_level)

{'Abusive': 0.03409159928560257, 'HS': 0.855967104434967}
{'HS_Group': 0.17336362600326538, 'HS_Individu': 0.7117328643798828}
{'HS_Physical': 5.453824996948242e-06, 'HS_Gender': 1.0222196578979492e-05, 'HS_Race': 0.00026670098304748535, 'HS_Religion': 0.0036801695823669434, 'HS_Other': 0.9714818596839905}
{'HS_Strong': 0.010158302262425423, 'HS_Weak': 0.20429293811321259, 'HS_Moderate': 0.517462432384491}
